In [ ]:
############################################################################
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
############################################################################
#  DISCLAIMER
# The code shared by CEs with customers must be limited to non-production
# demo/example code or infrastructure automation scripts.
############################################################################
## Colab: @martincabrera, @scavaletto

# Vertex AI SDK for Python: applied examples **chat-bison model and BigQuery data (query)**
<table align="left">
  <td>
    <a href=" ">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href=" ">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
<a href=" " target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Installing the package

In [ ]:
# We have to first authenticate
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
!pip install google-cloud-aiplatform
!pip install google-api-python-client
!pip install langchain
!pip install google-search-results

! ^^^^ Do not forget to click the "Restart runtime" button above.


## Authenticating

In [ ]:
PROJECT_ID = "cloud-llm-preview1"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
DATASET_ID = "genai_demos"  # @param {type:"string"}
TABLE_ID = "supermarket_sales"  # @param {type:"string"}
MODEL_ID = "text-bison"  # @param {type:"string"}


In [ ]:
# Initialize the VertexAI client
vertexai.init(project=PROJECT_ID)

# Librerias Langchain + VertexAI

In [ ]:
# Cloud Run Service that implements the Google Cloud
# bot logic using the Google PaLM language model and the
# LangChain SQL llm chain.
#
import os
from flask import Flask
from flask import request
from flask_cors import CORS

from sqlalchemy.engine import create_engine
from sqlalchemy.schema import MetaData

from langchain.llms import VertexAI
from langchain.chat_models import ChatVertexAI
from langchain import SQLDatabase, SQLDatabaseChain
import vertexai

Chatbot code

In [ ]:
# Main Flask app
app = Flask(__name__)
CORS(app)

# Main chat endpoint
@app.route("/chat", methods=['POST'])
def chat():
    # Get the jsonrequest
    request_json = request.get_json(silent=True)

    # Get the input message from Google Chat
    message = request_json["message"]["text"]

    # Create de SQL engine
    project_id = PROJECT_ID
    dataset_id = DATASET_ID
    table_uri = f"bigquery://{project_id}/{dataset_id}"
    engine = create_engine(f"bigquery://{project_id}/{dataset_id}")
    engine.execute('SELECT * FROM genai_demos.supermarket_sales').first()

    # Create the SQL Database LangChain wrapper
    db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=[TABLE_ID])

    # Create the llm model reference
    llm = VertexAI(
        model_name=MODEL_ID,
        temperature=0
    )

    # Creates the LanChain db chain
    db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

    # Perform the language model completion
    llm_answer = db_chain.run(message)
    print(llm_answer)

    # Return the answer
    response = {
        "text": llm_answer
    }
    return response

# Run the app
if __name__ == "__main__":
    app.run(debug=True, host="0.0.0.0", port=int(os.environ.get("PORT", 8080)))